In [38]:
import duckdb
import pandas as pd
import numpy as np

from itertools import product
from datetime import datetime



In [39]:
import os
# Get the current working directory
cwd = os.getcwd()
print(cwd)
# Construct the desired file path
file_path = os.path.join(cwd,'interim', 'mydb.db')
print(file_path)

/var/home/fcabello/git/Duckdb-Fastapi-Nextjs-Dashboard-Example/01data
/var/home/fcabello/git/Duckdb-Fastapi-Nextjs-Dashboard-Example/01data/interim/mydb.db


In [40]:
db = duckdb.connect()

companies_df = pd.read_csv('../01data/raw/organization_and_zones_dataset.csv')



timeseries_dataset_df = pd.read_csv('../01data/raw/timeseries_dataset.csv')


# Convert the 'timestamp' and 'ingestion_time' columns to datetime format
timeseries_dataset_df['timestamp'] = pd.to_datetime(timeseries_dataset_df['timestamp']).dt.date
timeseries_dataset_df['ingestion_time'] = pd.to_datetime(timeseries_dataset_df['ingestion_time']).dt.date



variables = timeseries_dataset_df['variable'].unique()
organizations = timeseries_dataset_df['organization'].unique()

every_org_var = product(organizations, variables)


companies_df["last7daysmean_CHL-01"] = np.nan
companies_df["last7daysmean_SPM-01"] = np.nan
companies_df["mean_CHL-01"] = np.nan
companies_df["mean_SPM-01"] = np.nan
companies_df["last7daysvar_CHL-01"] = np.nan
companies_df["last7daysvar_SPM-01"] = np.nan
companies_df["nullto_nonnull_CHL-01"] = np.nan
companies_df["nullto_nonnull_SPM-01"] = np.nan
companies_df["first_day"] = np.nan
companies_df["last_day"] = np.nan



subset_df = None
for org, var in every_org_var:
    subset_df = timeseries_dataset_df[(timeseries_dataset_df["organization"] == org) 
                          & (timeseries_dataset_df["variable"] == var)]
    
    companies_df.loc[companies_df['organization'] == org, f'last7daysmean_{var}'] = subset_df.sort_values(by='timestamp', ascending=False)[:7]['value'].mean()
    companies_df.loc[companies_df['organization'] == org, f'last7daysvar_{var}'] = subset_df.sort_values(by='timestamp', ascending=False)[:7]['value'].var()
    
    companies_df.loc[companies_df['organization'] == org, f'mean_{var}'] = subset_df['value'].mean()
    
    companies_df.loc[companies_df['organization'] == org, f'nullto_nonnull_{var}'] = subset_df['value'].isnull().sum() / subset_df['value'].count()
    

# Create the DuckDB table
db.execute("CREATE TABLE companies AS SELECT * FROM companies_df")
db.execute("""
    CREATE TABLE timeseries_dataset AS
    SELECT
        timestamp,
        variable,
        organization,
        value,
        ingestion_time
    FROM timeseries_dataset_df
""")


db.execute(f"EXPORT DATABASE '{file_path}'")

db.close()

datetime.date(2024, 4, 13)

In [41]:
timeseries_dataset_df

,timestamp,variable,organization,value,ingestion_time
0,2024-04-13,CHL-01,adasa,1.451352,2024-04-14
1,2024-04-13,SPM-01,adasa,3.337477,2024-04-14
2,2024-04-13,CHL-01,gsinima,1.010194,2024-04-14
3,2024-04-13,SPM-01,gsinima,NaN,2024-04-14
4,2024-04-12,SPM-01,adasa,0.949812,2024-04-13
...,...,...,...,...,...
685,2023-11-08,SPM-01,gsinima,0.648428,2023-11-11
686,2023-11-07,CHL-01,adasa,0.766285,2023-11-11
687,2023-11-07,CHL-01,gsinima,2.131320,2023-11-11
688,2023-11-07,SPM-01,adasa,NaN,2023-11-11


In [42]:
companies_df

,organization,zone_id,zone,polygon_decoded,last7daysmean_CHL-01,last7daysmean_SPM-01,mean_CHL-01,mean_SPM-01,last7daysvar_CHL-01,last7daysvar_SPM-01,nullto_nonnull_CHL-01,nullto_nonnull_SPM-01
0,gsinima,25,caldera,"-70.87416271129088,-27.014335684429216;-70.834...",0.895229,0.249040,2.299342,0.774969,0.282359,0.019374,0.035503,0.240876
1,adasa,27,antofagasta,"-70.42940139770508,-23.534481206739827;-70.431...",5.358125,1.859216,3.008599,1.451725,34.547261,0.780885,0.035503,0.205674


  Obtaining dependency information for pretty-errors from https://files.pythonhosted.org/packages/6d/8e/2df7467a15eae40e26c476683962fdb810cd1b36676603e2f139b4abbeaf/pretty_errors-1.2.25-py3-none-any.whl.metadata
  Obtaining dependency information for colorama from https://files.pythonhosted.org/packages/d1/d6/3965ed04c63042e047cb6a3e6ed1a63a35087b6a609aa3a15ed8ac56c221/colorama-0.4.6-py2.py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
